In [ ]:
# Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Install Python libraries
!pip install --upgrade ollama pypdf sentence-transformers chromadb

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import os
import asyncio

# Function to run shell commands
async def run_command(command):
    process = await asyncio.create_subprocess_shell(
        command,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )
    stdout, stderr = await process.communicate()
    if process.returncode != 0:
        print(f"Error: {stderr.decode().strip()}")
    return stdout.decode().strip()

# Start Ollama server in the background
# Using os.setsid() to ensure it runs in a new session and doesn't terminate
pid = os.fork()
if pid == 0:
    os.setsid()
    os.execvp('ollama', ['ollama', 'serve'])
else:
    print("Ollama server started in the background.")

# Give the server a moment to start
!sleep 5

# Pull the model
print("Pulling llama3.2:1b model. This may take a few minutes...")
!ollama pull llama3.2:1b
print("Model pulled successfully.")

/tmp/ipython-input-2353384273.py:18: DeprecationWarning: This process (pid=700) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()


Ollama server started in the background.
Pulling llama3.2:1b model. This may take a few minutes...

Model pulled successfully.


In [ ]:
import chromadb
import uuid
import ollama
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer

# Your ChromaDB Cloud connection details
CHROMA_API_KEY = 'your_key'
CHROMA_TENANT = 'your_key'
CHROMA_DATABASE = 'Rag-Chat'

In [ ]:
from google.colab import files
import io

# A simple function to split text into chunks
def chunk_text(text, chunk_size=1000, chunk_overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - chunk_overlap
    return chunks

# Upload the PDF file
print("Please upload your PDF file.")
uploaded = files.upload()

if not uploaded:
    print("\nNo file uploaded. Please run the cell again and select a file.")
else:
    pdf_name = next(iter(uploaded))
    print(f"\nProcessing '{pdf_name}'...")

    # Read the text from the uploaded PDF
    reader = PdfReader(pdf_name)
    full_document_text = "\n".join(page.extract_text() for page in reader.pages)

    # Split the text into chunks
    text_chunks = chunk_text(full_document_text)
    print(f"Document split into {len(text_chunks)} chunks.")

Please upload your PDF file.


Saving Manav_Soni_AI_Journey_Expanded.pdf to Manav_Soni_AI_Journey_Expanded.pdf

Processing 'Manav_Soni_AI_Journey_Expanded.pdf'...
Document split into 7 chunks.


In [ ]:
# Connect to ChromaDB Cloud
print("Connecting to ChromaDB Cloud...")
client = chromadb.CloudClient(
  api_key=CHROMA_API_KEY,
  tenant=CHROMA_TENANT,
  database=CHROMA_DATABASE
)

# Load the embedding model
print("Loading embedding model...")
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Create or get the collection
collection_name = "my_rag_collection_ollama"
print(f"Creating or getting collection: '{collection_name}'")
collection = client.get_or_create_collection(
    name=collection_name,
    embedding_function=None # We will provide embeddings manually
)

# Generate embeddings for each chunk
print("Creating embeddings for document chunks...")
chunk_embeddings = embedding_model.encode(text_chunks, show_progress_bar=True).tolist()

# Create unique IDs for each chunk
chunk_ids = [str(uuid.uuid4()) for _ in text_chunks]

# Add the documents to the collection
print("Adding documents to ChromaDB collection...")
collection.add(
    ids=chunk_ids,
    embeddings=chunk_embeddings,
    documents=text_chunks
)

print(f"Collection '{collection_name}' is ready with {collection.count()} documents.")

Connecting to ChromaDB Cloud...
Loading embedding model...
Creating or getting collection: 'my_rag_collection_ollama'
Creating embeddings for document chunks...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Adding documents to ChromaDB collection...
Collection 'my_rag_collection_ollama' is ready with 180 documents.


In [ ]:
def retrieve_context(query, top_k=5):
    """Queries the ChromaDB collection for the most relevant text chunks."""
    query_embedding = embedding_model.encode(query).tolist()
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    return results['documents'][0]

def format_prompt(context, question):
    """Formats the prompt for the Ollama model."""
    # Ollama uses a messages format, so we create a system and user message
    system_prompt = f"""You are a helpful question-answering assistant. Use the provided context to answer the user's question. If you are unsure, say you don't know. Keep your response concise.

    Context:
    {context}
    """
    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': question}
    ]

# --- Test with a static question ---
static_question = "projects done by manav?" # You can change this question

print(f"Asking the static question: '{static_question}'")
print("-" * 30)

# 1. Retrieve relevant context from ChromaDB
retrieved_chunks = retrieve_context(static_question)
context_for_prompt = "\n---\n".join(retrieved_chunks)

# 2. Format the final prompt
messages = format_prompt(context_for_prompt, static_question)

# 3. Get the response from the local Ollama model
response = ollama.chat(
    model='llama3.2:1b',
    messages=messages
)

# 4. Extract and print the final answer
final_answer = response['message']['content']

print("Answer:")
print(final_answer)

Asking the static question: 'projects done by manav?'
------------------------------
Answer:
Manav Soni has mentioned several projects in his portfolio and online presence. Here are a few:

1. **YOLOv11 License Plate Recognition**: As I mentioned earlier, Manav's project is to detect two-wheeler license plates using YOLOv11m.
2. **Midjourney image generation**: Manav uses Midjourney to generate AI images of various objects and scenes, including a tight rope between two airships and a person sitting on a chair with a mahogany table in front of them.

These projects showcase Manav's interests in artificial intelligence, machine learning, and data analysis.
